In [1]:
# it works for disease progression program for Healthy and Lesion
import scanpy as sc
import numpy as np
import pandas as pd
import sys
from collections import Counter

In [2]:
def compute_contamination(adata, ctlabel):
    contamination = {}
    for ct in set(adata.obs[ctlabel]):
        scores = pd.DataFrame(adata.uns['rank_genes_groups']['scores'])[ct]
        names = pd.DataFrame(adata.uns['rank_genes_groups']['names'])[ct]
        threshold = np.mean(scores) - 6*np.std(scores)
        contamination_genes = names[scores<threshold]
        contamination[ct] = contamination_genes
    return contamination

In [3]:
def compute_diseaseprogression_programs(filename, tissue, patientkey, celltypelabel, diagnosislabel, healthylabel, diseaselabel):
    diseaselabel_mapping = {healthylabel:"Healthy", diseaselabel:"Disease"}
    adata = sc.read(filename)
    for ctlabel in [celltypelabel]:
        subset = adata[adata.obs[diagnosislabel]==diseaselabel]
        sc.tl.rank_genes_groups(subset, 
                                use_raw=False,   #make sure use adata.X, not adata.raw.X
                                groupby=ctlabel, 
                                reference='rest', 
                                n_genes=subset.shape[1], 
                                method='wilcoxon')
        
        adata.uns['contamination_'+ctlabel] = compute_contamination(subset, celltypelabel)
        adata.obs['DEstatus'] = [diseaselabel_mapping.get(diagnosis, 'Unknown') + '_' + ct for diagnosis, ct in zip(adata.obs[diagnosislabel], adata.obs[ctlabel])]
        destatus_counts = Counter(adata.obs['DEstatus'])
        discard = False
        for ct in set(adata.obs[ctlabel]):
            discard = False
            for k in ['Healthy_'+ct, 'Disease_'+ct]:
                if destatus_counts.get(k, 0) < 5:
                    discard = True
            print(ct, destatus_counts.get('Healthy_'+ct), destatus_counts.get('Disease_'+ct), discard)
            if discard:
                continue
            sc.tl.rank_genes_groups(adata, 
                                    use_raw=False,
                                    groupby='DEstatus', 
                                    reference='Healthy_'+ct, 
                                    groups=['Disease_'+ct], 
                                    key_added=ct+'_DE', 
                                    n_genes=adata.shape[1], 
                                    method='wilcoxon'
                                   )
        write_diseaseprogression_matrix(adata, 
                                          filedir,
                                          tissue,
                                          celltypelabel)

In [4]:
def write_diseaseprogression_matrix(adata, filedir, filename, celltypelabel):
    # set up the ordering of genes and cells
    #adata = adatas[filename]
    genes = list(set(adata.var_names))
    gene2idx = {gene:i for i, gene in enumerate(genes)}
    
    pvalmtxs, logfoldmtxs, scoremtxs = [], [], []
    ctlabels = [col for col in adata.uns if '_DE' in col]
    print(adata.obs.columns)
    print(ctlabels)
    for delabel in ctlabels:
        #delabel = ctlabel + '_DE'
        ct = delabel.split('_')[0]
        contamination = adata.uns['contamination_'+celltypelabel].get(ct, np.array([])).tolist()
        
        cellsubsets = adata.uns[delabel]['names'].dtype.fields.keys()
        cell2idx = {cellsubset:i for i, cellsubset in enumerate(cellsubsets)}

        # create empty matrix
        pvalmtx = np.zeros((len(gene2idx), len(cell2idx)))

        logfoldmtx = np.zeros((len(gene2idx), len(cell2idx)))
        scoremtx = np.zeros((len(gene2idx), len(cell2idx)))

        # loop through and fill up the matrix with pvalue, logfold and score
        for gene, pval, logfold, score in zip(adata.uns[delabel]['names'], 
                                       adata.uns[delabel]['pvals_adj'], 
                                       adata.uns[delabel]['logfoldchanges'], 
                                       adata.uns[delabel]['scores']):
            for cell_subset in cellsubsets:
                
                if gene[cell_subset] in contamination:
                    p = 1
                    l = 0
                    s = 0
                else:
                    p = pval[cell_subset]
                    l = logfold[cell_subset]
                    s = score[cell_subset]
                
                if gene[cell_subset] in gene2idx:
                    pvalmtx[gene2idx[gene[cell_subset]], cell2idx[cell_subset]] = p
                    logfoldmtx[gene2idx[gene[cell_subset]], cell2idx[cell_subset]] = l
                    scoremtx[gene2idx[gene[cell_subset]], cell2idx[cell_subset]] = s

        # transform matrix to dataframe
        cellsubsets = [ct for ct in cellsubsets]
        pvalmtxs.append(pd.DataFrame(pvalmtx, index=genes, columns=cellsubsets))
        logfoldmtxs.append(pd.DataFrame(logfoldmtx, index=genes, columns=cellsubsets))
        scoremtxs.append(pd.DataFrame(scoremtx, index=genes, columns=cellsubsets))
    pvalmtxs = pd.concat(pvalmtxs, axis=1)
    logfoldmtxs = pd.concat(logfoldmtxs, axis=1)
    scoremtxs = pd.concat(scoremtxs, axis=1)

    # write matrix to file
    pvalmtxs.to_csv("%s/%s_pval.csv"%(filedir, filename))
    logfoldmtxs.to_csv("%s/%s_logfold.csv"%(filedir, filename))
    scoremtxs.to_csv("%s/%s_score.csv"%(filedir, filename))

In [5]:
# check the input data:
# 1. Make sure cell_type and other values are STRING,not Integer
# 2. Check adata.X, adata.raw.X. Make suer adata.X is the log normalized data

In [6]:
adata_test = sc.read("/projects/abv/users/tangfx1/scRNAseq_data/hca_skin_portal/HCA_AD_Lesion_vs_Healthy_level1.h5ad")
adata_test.obs 

,orig.ident,nCount_RNA,nFeature_RNA,sample_id,mad_prd,Status,Site,Tissue,Enrichment,Location,Sex,Age,stage,full_clustering,patient_ID,celltype_level1,celltype_level2
AAACCTGAGAGGGCTT-1-SKN8090528,SeuratProject,2210.0,967,SKN8090528,0,Eczema,lesion,Epidermis,CD45P,Lower_back,Female,62.0,adult,Tc,E2,T_cells,Tc
AAACGGGAGAAGGACA-1-SKN8090528,SeuratProject,2637.0,1080,SKN8090528,0,Eczema,lesion,Epidermis,CD45P,Lower_back,Female,62.0,adult,Tc,E2,T_cells,Tc
AAACGGGTCAATCACG-1-SKN8090528,SeuratProject,1731.0,787,SKN8090528,0,Eczema,lesion,Epidermis,CD45P,Lower_back,Female,62.0,adult,Tc,E2,T_cells,Tc
AAAGATGCACATGGGA-1-SKN8090528,SeuratProject,2885.0,1026,SKN8090528,0,Eczema,lesion,Epidermis,CD45P,Lower_back,Female,62.0,adult,moDC_1,E2,Dendritic_cells,moDC_1
AAAGATGCACCGAATT-1-SKN8090528,SeuratProject,1563.0,752,SKN8090528,0,Eczema,lesion,Epidermis,CD45P,Lower_back,Female,62.0,adult,Tc,E2,T_cells,Tc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTTCAGGCCCA-1-4820STDY7389014,SeuratProject,1023.0,455,4820STDY7389014,0,Healthy,non_lesion,Epidermis,Lymphocytes,Breast,Female,NaN,adult,LC_2,s3,Langerhans_cell,Langerhans_cell_2
TTTGGTTTCGCCTGTT-1-4820STDY7389014,SeuratProject,4430.0,1228,4820STDY7389014,0,Healthy,non_lesion,Epidermis,Lymphocytes,Breast,Female,NaN,adult,Th,s3,T_cells,Th
TTTGTCAAGGAATCGC-1-4820STDY7389014,SeuratProject,2633.0,739,4820STDY7389014,0,Healthy,non_lesion,Epidermis,Lymphocytes,Breast,Female,NaN,adult,Th,s3,T_cells,Th
TTTGTCAAGGACTGGT-1-4820STDY7389014,SeuratProject,3394.0,1390,4820STDY7389014,0,Healthy,non_lesion,Epidermis,Lymphocytes,Breast,Female,NaN,adult,Th,s3,T_cells,Th


In [6]:
print(adata_test.X)

  (0, 27)	2.3075501223766746
  (0, 55)	1.709262772592952
  (0, 89)	1.709262772592952
  (0, 154)	2.679284447813446
  (0, 201)	1.709262772592952
  (0, 361)	1.709262772592952
  (0, 493)	3.6162844336343807
  (0, 503)	1.709262772592952
  (0, 526)	2.3075501223766746
  (0, 531)	2.3075501223766746
  (0, 546)	1.709262772592952
  (0, 559)	2.3075501223766746
  (0, 618)	1.709262772592952
  (0, 682)	1.709262772592952
  (0, 683)	1.709262772592952
  (0, 733)	1.709262772592952
  (0, 741)	1.709262772592952
  (0, 752)	1.709262772592952
  (0, 761)	1.709262772592952
  (0, 771)	1.709262772592952
  (0, 918)	3.3375232429972668
  (0, 935)	2.3075501223766746
  (0, 953)	2.3075501223766746
  (0, 977)	2.3075501223766746
  (0, 1019)	1.709262772592952
  :	:
  (259250, 32829)	1.3565243806290652
  (259250, 32840)	1.3565243806290652
  (259250, 32868)	1.3565243806290652
  (259250, 33056)	1.3565243806290652
  (259250, 33057)	1.3565243806290652
  (259250, 33082)	1.3565243806290652
  (259250, 33207)	1.3565243806290652
  (

In [7]:
print(adata_test.raw.X)

  (0, 27)	2.0
  (0, 55)	1.0
  (0, 89)	1.0
  (0, 154)	3.0
  (0, 201)	1.0
  (0, 361)	1.0
  (0, 493)	8.0
  (0, 503)	1.0
  (0, 526)	2.0
  (0, 531)	2.0
  (0, 546)	1.0
  (0, 559)	2.0
  (0, 618)	1.0
  (0, 682)	1.0
  (0, 683)	1.0
  (0, 733)	1.0
  (0, 741)	1.0
  (0, 752)	1.0
  (0, 761)	1.0
  (0, 771)	1.0
  (0, 918)	6.0
  (0, 935)	2.0
  (0, 953)	2.0
  (0, 977)	2.0
  (0, 1019)	1.0
  :	:
  (259250, 32829)	1.0
  (259250, 32840)	1.0
  (259250, 32868)	1.0
  (259250, 33056)	1.0
  (259250, 33057)	1.0
  (259250, 33082)	1.0
  (259250, 33207)	1.0
  (259250, 33249)	2.0
  (259250, 33252)	1.0
  (259250, 33296)	2.0
  (259250, 33326)	2.0
  (259250, 33359)	1.0
  (259250, 33445)	1.0
  (259250, 33446)	1.0
  (259250, 33459)	1.0
  (259250, 33493)	1.0
  (259250, 33497)	1.0
  (259250, 33498)	9.0
  (259250, 33499)	3.0
  (259250, 33501)	1.0
  (259250, 33502)	2.0
  (259250, 33503)	1.0
  (259250, 33505)	3.0
  (259250, 33506)	1.0
  (259250, 33508)	1.0


In [8]:
adata_test

AnnData object with n_obs × n_vars = 259179 × 33538
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample_id', 'mad_prd', 'Status', 'Site', 'Tissue', 'Enrichment', 'Location', 'Sex', 'Age', 'stage', 'full_clustering', 'patient_ID', 'celltype_level1', 'celltype_level2'
    var: 'features'

In [9]:
#compute_diseaseprogression_programs:
#compute_diseaseprogression_programs(filename, 
#                                            tissue, 
#                                            sampleid, 
#                                            celltypelabel, 
#                                            diagnosislabel, 
#                                            healthylabel, 
#                                            diseaselabel
#                                           )"
# for 14 clusters:
filedir = '/projects/abv/users/tangfx1/scRNAseq_data/hca_skin_portal/AD_HCA_scanpy_output/14_clusters_AD_Lesion_vs_HC_DP/' #the output dir
filename = 'HCA_AD_Lesion_vs_Healthy_level1.h5ad' #the input and output file name
compute_diseaseprogression_programs(filename,
                                    'skin',
                                    'sampleid',
                                    'celltype_level1',
                                    'Status',
                                    'Healthy',
                                    'Eczema'
                                   )




/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'orig.ident' as categorical
/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
Trying to set attribute `.obs` of view, copying.
...

Lymphatic_endothelium 4924 789 False


/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'orig.ident' as categorical
/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'sample_id' as categorical
/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Re

Langerhans_cell 12545 2419 False
Macrophages 13890 4165 False
Melanocytes 3682 596 False
ILC_NK_cells 6772 541 False
Schwann_cells 269 53 False
Mast_cells 557 94 False
Fibroblasts 17456 10236 False
Pericytes 5006 3901 False
T_cells 35040 18159 False
Vascular_endothelium 25785 5338 False
Keratinocytes 53844 11475 False
Dendritic_cells 15898 5745 False
Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample_id', 'mad_prd',
       'Status', 'Site', 'Tissue', 'Enrichment', 'Location', 'Sex', 'Age',
       'stage', 'full_clustering', 'patient_ID', 'celltype_level1',
       'celltype_level2', 'DEstatus'],
      dtype='object')
['Lymphatic_endothelium_DE', 'Langerhans_cell_DE', 'Macrophages_DE', 'Melanocytes_DE', 'ILC_NK_cells_DE', 'Schwann_cells_DE', 'Mast_cells_DE', 'Fibroblasts_DE', 'Pericytes_DE', 'T_cells_DE', 'Vascular_endothelium_DE', 'Keratinocytes_DE', 'Dendritic_cells_DE']


In [10]:
# for 41 clusters:
filedir = '/projects/abv/users/tangfx1/scRNAseq_data/hca_skin_portal/AD_HCA_scanpy_output/41_clusters_AD_Lesion_vs_HC_DP/' #the output dir
filename = 'HCA_AD_Lesion_vs_Healthy_level2.h5ad' #the input and output file name
compute_diseaseprogression_programs(filename,
                                    'skin',
                                    'sampleid',
                                    'celltype_level2',
                                    'Status',
                                    'Healthy',
                                    'Eczema'
                                   )




/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'orig.ident' as categorical
/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
Trying to set attribute `.obs` of view, copying.
...

Schwann_2 120 7 False


/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'orig.ident' as categorical
/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'sample_id' as categorical
/ui/abv/tangfx1/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Re

ILC2 494 53 False
Lymphatic_endothelium_1 1073 533 False
moDC_3 3171 376 False
Mast_cell 557 94 False
moDC_2 4362 342 False
Differentiated_KC 25418 8267 False
Vascular_endothelium_3 629 621 False
MigDC 2080 827 False
Treg 5743 2875 False
Fibroblast_1 10161 1148 False
Macrophage_1 5781 339 False
DC1 604 502 False
Pericyte_1 4705 1907 False
DC2 724 2955 False
Macrophage_2 1424 478 False
Langerhans_cell_2 4205 515 False
Proliferating_KC 1803 662 False
Mono_Macrophage 4912 628 False
Langerhans_cell_3 2542 873 False
Inflam_Macrophage 1773 2720 False
Fibroblast_3 3035 1345 False
Fibroblast_2 4260 7743 False
Schwann_1 149 46 False
Th 20617 6173 False
moDC_1 4957 743 False
Langerhans_cell_4 5230 138 False
Tc 8680 5935 False
ILC1_3 2935 215 False
Pericyte_2 301 1994 False
Melanocyte 3682 596 False
Undifferentiated_KC 21348 1336 False
Lymphatic_endothelium_2 3851 256 False
Langerhans_cell_1 568 893 False
Vascular_endothelium_1 9645 1627 False
Vascular_endothelium_2 15511 3090 False
NK 739 273 Fa